In [192]:
import urllib.request as req
from bs4 import BeautifulSoup as bs
import pandas as pd
import pymysql
import json
import urllib.parse

In [193]:
key = "PLne6yt5IwfGITmjviWt3jgM6Q9gEeTGcZAu5KSq6yrlhLJedSV56NzrMxChqpwf7usIHfQtO%2BpvJYInNXRTyw%3D%3D"

In [194]:
# 노선 정보 xml 파일 수집
busNo = input("busNo")
url1 = "http://apis.data.go.kr/6260000/BusanBIMS/busInfo"
params1 = f"?serviceKey={key}&lineno={busNo}"
save_path1 = "../data/공공DB/버스노선정보.xml"
req.urlretrieve(url1+params1, save_path1)

('../data/공공DB/버스노선정보.xml', <http.client.HTTPMessage at 0x2b731de1d30>)

In [195]:
# xml 파일 파싱
xml = open(save_path1, "r", encoding="utf-8")
soup = bs(xml, "xml")

# 버스 번호의 노선 ID 추출
items = soup.find_all("item")
buslinenum = ""
lineid = ""
for item in items:
    buslinenum = item.find("buslinenum").text
    if buslinenum == str(busNo):
        lineid = item.find("lineid").text
print(lineid)

5200138000


In [196]:
# 노선 정류소 정보 xml 파일 수집
url2 = "http://apis.data.go.kr/6260000/BusanBIMS/busInfoByRouteId"
params2 = f"?serviceKey={key}&lineid={lineid}"
save_path2 = "../data/공공DB/노선 정류소 조회 정보.xml"

req.urlretrieve(url2+params2, save_path2)

('../data/공공DB/노선 정류소 조회 정보.xml', <http.client.HTTPMessage at 0x2b7319fc1d0>)

In [197]:
# 노선 정류소 정보 xml 파일 처리
xml = open(save_path2, "r", encoding="utf-8")
soup = bs(xml, "xml")
items = soup.find_all("item")
stopNames = []
for item in items:
    direction = item.find("direction")
    if direction != None:
        bstopnm = item.find("bstopnm").text
        lat = item.find("lat").text
        lin =  item.find("lin").text
        stopNames.append(bstopnm)
        print((bstopnm, lat, lin))

('신평배고개', '35.0962720', '128.9780864')
('사하구청.당리역', '35.1040928', '128.9725056')
('엄궁동주민센터', '35.1307136', '128.9717504')
('주물공단입구', '35.1512800', '128.9782400')
('주례역', '35.1505824', '129.0034688')
('가야역', '35.1565632', '129.0456448')
('중앙시장', '35.1433024', '129.0591104')
('문현현대아파트', '35.1442208', '129.0766336')
('대연SK뷰아파트', '35.1394880', '129.0800000')
('유엔공원.부산문화회관', '35.1265056', '129.0955008')
('신선대CNG충전소', '35.1139648', '129.0963456')
('대연역.부산은행', '35.1334816', '129.0920704')
('문현현대아파트', '35.1439360', '129.0769152')
('중앙시장', '35.1428160', '129.0592768')
('부산진우체국(가야1치안센터)', '35.1550816', '129.0373632')
('개금역', '35.1531968', '129.0244352')
('감전교차로', '35.1427808', '128.9697664')
('에덴공원', '35.1106496', '128.9654272')
('사남신익아파트', '35.0966656', '128.9838848')
('장림', '35.0849568', '128.9792512')
('기점', '35.0847744', '128.9796352')


In [198]:
# 정류소 정보 조회 xml 파일 수집
url3 = "http://apis.data.go.kr/6260000/BusanBIMS/busStopList"
params3 = f"?serviceKey={key}&bstopnm={urllib.parse.quote(stopNames[0])}"
save_path3 = "../data/공공DB/정류소정보조회.xml"

res = req.urlretrieve(url3+params3, save_path3)

In [199]:
# 정류소 정보 조회 xml 파일 처리
xml = open(save_path3, "r", encoding="utf-8")
soup = bs(xml, "xml")
stopIDs = []
for item in soup.find_all("item"):
    stoptype = item.find("stoptype").text
    if stoptype == "일반":
        bstopid = item.find("bstopid").text
        stopIDs.append(bstopid)

print(stopIDs)

['188090203', '188090402']


In [200]:
# 정류소 도착정보 조회 xml 파일 수집
for stopID in stopIDs:
    url4 = "http://apis.data.go.kr/6260000/BusanBIMS/stopArrByBstopid"
    params4 = f"?serviceKey={key}&bstopid={stopID}"
    xml = req.urlopen(url4+params4).read().decode("utf-8")
    soup = bs(xml, "xml")
    for item in soup.find_all("item"):
        nodenm = item.find("nodenm").text
        lineno = item.find("lineno").text
        if (lineno == busNo):
            try:
                min1 = item.find("min1").text
            except:
                min1 = "-"
            try:
                station1 = item.find("station1").text
            except:
                station1 = "-"
            print(f"{nodenm} 정류장의 {lineno}번 버스 도착까지 {min1}분 전이며, 남은 정류장은 {station1}곳입니다.")


신평배고개 정류장의 138번 버스 도착까지 -분 전이며, 남은 정류장은 -곳입니다.
신평배고개 정류장의 138번 버스 도착까지 3분 전이며, 남은 정류장은 2곳입니다.
